# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d200/7037ec05b97576da1bd9db5e3ea6a322a577bc833b0dc772238248ef.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Toronto, Ontario, Canada**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(200,'7037ec05b97576da1bd9db5e3ea6a322a577bc833b0dc772238248ef')

# Data Processing

In [18]:
import pandas as pd

# read csv
weather_df = pd.read_csv('data/C2A2_data/BinnedCsvs_d200/7037ec05b97576da1bd9db5e3ea6a322a577bc833b0dc772238248ef.csv')

# process weather data - drop id, sort by date
weather_df = weather_df.drop(['ID'], axis=1).sort_values(by='Date')#.set_index('Date')
# compute lowest daily tempareture across the region
minTemp_df = weather_df[weather_df['Element']=='TMIN'].drop(['Element'], axis=1)
minTemp_df = minTemp_df.groupby('Date').agg({"Data_Value":[min]})
minTemp_df.columns = ['TMIN']
# compute highest daily tempareture across the region
maxTemp_df = weather_df[weather_df['Element']=='TMAX'].drop(['Element'], axis=1)
maxTemp_df = maxTemp_df.groupby('Date').agg({"Data_Value":[max]})
maxTemp_df.columns = ['TMAX']

# temperature df by date
temperature_df = pd.concat([minTemp_df,maxTemp_df], axis=1, join='inner').reset_index()

#import datetime
# convert string to timestamp
temperature_df['Date'] = pd.to_datetime(temperature_df['Date'])#.dt.date #, format='%d%b%Y:%H:%M:%S.%f')
type(temperature_df['Date'].iloc[0])
temperature_df['Year'] = temperature_df['Date'].dt.strftime('%Y')
temperature_df['MonthDay'] = temperature_df['Date'].dt.strftime('%m-%d')
#temperature_df['DayOfYear'] = temperature_df['Date'].dt.strftime('%j').map(int)

# remove leap day: delete month day of 02-29
temperature_df = temperature_df[(temperature_df['MonthDay']!='02-29')]
temperature_df

Date  TMIN  TMAX  Year MonthDay
0    2005-01-01   -47   128  2005    01-01
1    2005-01-02   -50   128  2005    01-02
2    2005-01-03   -45   161  2005    01-03
3    2005-01-04   -40    67  2005    01-04
4    2005-01-05  -100    50  2005    01-05
5    2005-01-06  -110    56  2005    01-06
6    2005-01-07  -100    44  2005    01-07
7    2005-01-08   -67    16  2005    01-08
8    2005-01-09   -55    50  2005    01-09
9    2005-01-10   -58    56  2005    01-10
10   2005-01-11   -70    30  2005    01-11
11   2005-01-12   -80   161  2005    01-12
12   2005-01-13   -51   206  2005    01-13
13   2005-01-14  -127   200  2005    01-14
14   2005-01-15  -145    11  2005    01-15
15   2005-01-16  -170   -30  2005    01-16
16   2005-01-17  -223   -33  2005    01-17
17   2005-01-18  -275   -30  2005    01-18
18   2005-01-19  -228   -10  2005    01-19
19   2005-01-20  -235    -6  2005    01-20
20   2005-01-21  -320   -83  2005    01-21
21   2005-01-22  -288   -81  2005    01-22
22   2005-01-23  -260   -60  2005    01-23
23   2005-01-24  -260   -10  2005    01-24
24   2005-01-25  -156     6  2005    01-25
25   2005-01-26  -251    11  2005    01-26
26   2005-01-27  -311   -70  2005    01-27
27   2005-01-28  -300   -35  2005    01-28
28   2005-01-29  -260    20  2005    01-29
29   2005-01-30  -200    22  2005    01-30
...         ...   ...   ...   ...      ...
3987 2015-12-02   -31   122  2015    12-02
3988 2015-12-03   -30   106  2015    12-03
3989 2015-12-04   -30    78  2015    12-04
3990 2015-12-05   -70    93  2015    12-05
3991 2015-12-06   -60   100  2015    12-06
3992 2015-12-07   -55   100  2015    12-07
3993 2015-12-08   -71   120  2015    12-08
3994 2015-12-09   -30   144  2015    12-09
3995 2015-12-10   -20   150  2015    12-10
3996 2015-12-11   -30   156  2015    12-11
3997 2015-12-12   -25   161  2015    12-12
3998 2015-12-13   -20   200  2015    12-13
3999 2015-12-14    35   222  2015    12-14
4000 2015-12-15   -13   222  2015    12-15
4001 2015-12-16   -30   117  2015    12-16
4002 2015-12-17   -20   117  2015    12-17
4003 2015-12-18   -53   122  2015    12-18
4004 2015-12-19   -72    72  2015    12-19
4005 2015-12-20   -82    90  2015    12-20
4006 2015-12-21   -70   120  2015    12-21
4007 2015-12-22   -44   160  2015    12-22
4008 2015-12-23    10   183  2015    12-23
4009 2015-12-24   -10   189  2015    12-24
4010 2015-12-25   -75   172  2015    12-25
4011 2015-12-26   -60   122  2015    12-26
4012 2015-12-27  -110   122  2015    12-27
4013 2015-12-28  -100   106  2015    12-28
4014 2015-12-29   -85    80  2015    12-29
4015 2015-12-30   -60    78  2015    12-30
4016 2015-12-31   -29    67  2015    12-31

[4015 rows x 5 columns]

In [19]:
# get final temperature by day dataframe
temperatureByDay_df = temperature_df.groupby('MonthDay').agg({"TMIN":[min], "TMAX":[max],}).reset_index()
temperatureByDay_df.columns = ['MonthDay', 'TMIN', 'TMAX']
temperatureByDay_df

MonthDay  TMIN  TMAX
0      01-01   134  -250
1      01-02   139  -245
2      01-03   161  -270
3      01-04   144  -222
4      01-05   150  -200
5      01-06   160  -260
6      01-07   178  -270
7      01-08   194  -260
8      01-09   183  -236
9      01-10   110  -229
10     01-11   150  -200
11     01-12   161  -250
12     01-13   206  -260
13     01-14   200  -290
14     01-15    89  -290
15     01-16   103  -260
16     01-17   128  -270
17     01-18   128  -275
18     01-19   117  -228
19     01-20   136  -250
20     01-21   139  -320
21     01-22   117  -306
22     01-23   106  -280
23     01-24   106  -295
24     01-25   106  -272
25     01-26    89  -260
26     01-27    83  -311
27     01-28   125  -300
28     01-29   170  -260
29     01-30   189  -255
..       ...   ...   ...
335    12-02   140  -135
336    12-03   170  -114
337    12-04   191  -130
338    12-05   194  -159
339    12-06   167  -180
340    12-07   100  -213
341    12-08   120  -214
342    12-09   144  -210
343    12-10   150  -185
344    12-11   156  -175
345    12-12   161  -220
346    12-13   200  -220
347    12-14   222  -206
348    12-15   222  -189
349    12-16   144  -230
350    12-17   150  -245
351    12-18   128  -167
352    12-19    83  -159
353    12-20   100  -230
354    12-21   144  -190
355    12-22   167  -180
356    12-23   183  -170
357    12-24   189  -230
358    12-25   172  -224
359    12-26   122  -194
360    12-27   178  -200
361    12-28   178  -160
362    12-29   172  -180
363    12-30   100  -170
364    12-31   133  -170

[365 rows x 3 columns]

# Plot Functions for time series data

In [148]:
def simpleLinePlot(data, y, x, ylabel, xlabel):
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure()

    languages =['Python', 'SQL', 'Java', 'C++', 'JavaScript']
    pos = np.arange(len(languages))
    popularity = [56, 39, 34, 34, 29]

    plt.bar(pos, popularity, align='center')
    plt.xticks(pos, languages)
    plt.ylabel('% Popularity')
    plt.title('Top 5 Languages for Math & Data \nby % Popularity on Stack Overflow')

    plt.show()
    

In [3]:
temperature_df

Date  TMIN  TMAX  Year MonthDay
0    2005-01-01   -47   128  2005    01-01
1    2005-01-02   -50   128  2005    01-02
2    2005-01-03   -45   161  2005    01-03
3    2005-01-04   -40    67  2005    01-04
4    2005-01-05  -100    50  2005    01-05
5    2005-01-06  -110    56  2005    01-06
6    2005-01-07  -100    44  2005    01-07
7    2005-01-08   -67    16  2005    01-08
8    2005-01-09   -55    50  2005    01-09
9    2005-01-10   -58    56  2005    01-10
10   2005-01-11   -70    30  2005    01-11
11   2005-01-12   -80   161  2005    01-12
12   2005-01-13   -51   206  2005    01-13
13   2005-01-14  -127   200  2005    01-14
14   2005-01-15  -145    11  2005    01-15
15   2005-01-16  -170   -30  2005    01-16
16   2005-01-17  -223   -33  2005    01-17
17   2005-01-18  -275   -30  2005    01-18
18   2005-01-19  -228   -10  2005    01-19
19   2005-01-20  -235    -6  2005    01-20
20   2005-01-21  -320   -83  2005    01-21
21   2005-01-22  -288   -81  2005    01-22
22   2005-01-23  -260   -60  2005    01-23
23   2005-01-24  -260   -10  2005    01-24
24   2005-01-25  -156     6  2005    01-25
25   2005-01-26  -251    11  2005    01-26
26   2005-01-27  -311   -70  2005    01-27
27   2005-01-28  -300   -35  2005    01-28
28   2005-01-29  -260    20  2005    01-29
29   2005-01-30  -200    22  2005    01-30
...         ...   ...   ...   ...      ...
3987 2015-12-02   -31   122  2015    12-02
3988 2015-12-03   -30   106  2015    12-03
3989 2015-12-04   -30    78  2015    12-04
3990 2015-12-05   -70    93  2015    12-05
3991 2015-12-06   -60   100  2015    12-06
3992 2015-12-07   -55   100  2015    12-07
3993 2015-12-08   -71   120  2015    12-08
3994 2015-12-09   -30   144  2015    12-09
3995 2015-12-10   -20   150  2015    12-10
3996 2015-12-11   -30   156  2015    12-11
3997 2015-12-12   -25   161  2015    12-12
3998 2015-12-13   -20   200  2015    12-13
3999 2015-12-14    35   222  2015    12-14
4000 2015-12-15   -13   222  2015    12-15
4001 2015-12-16   -30   117  2015    12-16
4002 2015-12-17   -20   117  2015    12-17
4003 2015-12-18   -53   122  2015    12-18
4004 2015-12-19   -72    72  2015    12-19
4005 2015-12-20   -82    90  2015    12-20
4006 2015-12-21   -70   120  2015    12-21
4007 2015-12-22   -44   160  2015    12-22
4008 2015-12-23    10   183  2015    12-23
4009 2015-12-24   -10   189  2015    12-24
4010 2015-12-25   -75   172  2015    12-25
4011 2015-12-26   -60   122  2015    12-26
4012 2015-12-27  -110   122  2015    12-27
4013 2015-12-28  -100   106  2015    12-28
4014 2015-12-29   -85    80  2015    12-29
4015 2015-12-30   -60    78  2015    12-30
4016 2015-12-31   -29    67  2015    12-31

[4017 rows x 5 columns]

In [95]:
temperature_df['Data_Value']['min']#.apply(index:index.strftime('%j'))

0       -47
1       -50
2       -45
3       -40
4      -100
5      -110
6      -100
7       -67
8       -55
9       -58
10      -70
11      -80
12      -51
13     -127
14     -145
15     -170
16     -223
17     -275
18     -228
19     -235
20     -320
21     -288
22     -260
23     -260
24     -156
25     -251
26     -311
27     -300
28     -260
29     -200
       ... 
3987    -31
3988    -30
3989    -30
3990    -70
3991    -60
3992    -55
3993    -71
3994    -30
3995    -20
3996    -30
3997    -25
3998    -20
3999     35
4000    -13
4001    -30
4002    -20
4003    -53
4004    -72
4005    -82
4006    -70
4007    -44
4008     10
4009    -10
4010    -75
4011    -60
4012   -110
4013   -100
4014    -85
4015    -60
4016    -29
Name: min, dtype: int64

In [85]:
temperature_df

Data_Value     
                  min  max
Date                      
2005-01-01        -47  128
2005-01-02        -50  128
2005-01-03        -45  161
2005-01-04        -40   67
2005-01-05       -100   50
2005-01-06       -110   56
2005-01-07       -100   44
2005-01-08        -67   16
2005-01-09        -55   50
2005-01-10        -58   56
2005-01-11        -70   30
2005-01-12        -80  161
2005-01-13        -51  206
2005-01-14       -127  200
2005-01-15       -145   11
2005-01-16       -170  -30
2005-01-17       -223  -33
2005-01-18       -275  -30
2005-01-19       -228  -10
2005-01-20       -235   -6
2005-01-21       -320  -83
2005-01-22       -288  -81
2005-01-23       -260  -60
2005-01-24       -260  -10
2005-01-25       -156    6
2005-01-26       -251   11
2005-01-27       -311  -70
2005-01-28       -300  -35
2005-01-29       -260   20
2005-01-30       -200   22
...               ...  ...
2015-12-02        -31  122
2015-12-03        -30  106
2015-12-04        -30   78
2015-12-05        -70   93
2015-12-06        -60  100
2015-12-07        -55  100
2015-12-08        -71  120
2015-12-09        -30  144
2015-12-10        -20  150
2015-12-11        -30  156
2015-12-12        -25  161
2015-12-13        -20  200
2015-12-14         35  222
2015-12-15        -13  222
2015-12-16        -30  117
2015-12-17        -20  117
2015-12-18        -53  122
2015-12-19        -72   72
2015-12-20        -82   90
2015-12-21        -70  120
2015-12-22        -44  160
2015-12-23         10  183
2015-12-24        -10  189
2015-12-25        -75  172
2015-12-26        -60  122
2015-12-27       -110  122
2015-12-28       -100  106
2015-12-29        -85   80
2015-12-30        -60   78
2015-12-31        -29   67

[4017 rows x 2 columns]

In [25]:
import matplotlib.pyplot as plt
import mplleaflet

In [27]:
import numpy as np

linear_data = np.array([1,2,3,4,5,6,7,8])
exponential_data = linear_data**2

plt.figure()
# plot the linear data and the exponential data
plt.plot(linear_data, '-o', exponential_data, '-o')
# plt.plot(linear_data, '-', exponential_data, '-')

In [21]:
plt.figure()